In [1]:
import pandas as pd
import numpy as np
import os
import shutil

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
#from sklearn.model_selection import train_test_split

from packaging import version
import pytorch_lightning as pl
from pytorch_lightning import Callback

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.optim as optimizers

from torchvision import datasets
from torchvision import transforms

import optuna
from optuna.integration import PyTorchLightningPruningCallback

In [2]:
if version.parse(pl.__version__) < version.parse("1.0.2"):
    raise RuntimeError("PyTorch Lightning>=1.0.2 is required for this example.")

In [3]:
all_data = pd.read_csv('../input/intern-compe4/all_for_nn.csv')
train_data = all_data[all_data["data_type"] == "train"]
train_data.head(3)

,id,goal,country,duration,category1,category2,html_content,state,data_type,region,...,category1_0,category1_6,category1_3,category1_4,category1_8,category1_9,category1_10,category1_1,category1_14,category1_2
0,0,4001-5000,4,28,12,143,"<div class=""contents""><div><span class=""bold"">...",0.0,train,2,...,0,0,0,0,0,0,0,0,0,0
1,1,3001-4000,16,33,5,108,"<div class=""contents""><div><h1 class=""page-anc...",0.0,train,2,...,0,0,0,0,0,0,0,0,0,0
2,2,19001-20000,21,29,7,122,"<div class=""contents""><div><p> As our society ...",0.0,train,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
print(train_data.columns[-20:])

Index(['country_19', 'country_11', 'country_13', 'country_0', 'country_14',
       'category1_12', 'category1_5', 'category1_7', 'category1_13',
       'category1_11', 'category1_0', 'category1_6', 'category1_3',
       'category1_4', 'category1_8', 'category1_9', 'category1_10',
       'category1_1', 'category1_14', 'category1_2'],
      dtype='object')


In [5]:
target_cols = ["state"]
feature_cols = ['duration', 'goal_min',
                '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24',
       'number_of_figure', 'number_of_paragraph',
       'length_of_text', 'country_4',
       'country_16', 'country_21', 'country_9', 'country_3', 'country_7',
       'country_8', 'country_5', 'country_1', 'country_12', 'country_15',
       'country_18', 'country_17', 'country_6', 'country_10', 'country_20',
       'country_2', 'country_19', 'country_11', 'country_13', 'country_0',
       'country_14',
        'category1_12', 'category1_5', 'category1_7', 'category1_13',
       'category1_11', 'category1_0', 'category1_6', 'category1_3',
       'category1_4', 'category1_8', 'category1_9', 'category1_10',
       'category1_1', 'category1_14', 'category1_2']


feature_cols = ['duration', 'goal_min',
                '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
       '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
       '43', '44', '45', '46', '47', '48', '49',
       'number_of_figure', 'number_of_video', 'number_of_paragraph',
       'length_of_text', 'country_4',
       'country_16', 'country_21', 'country_9', 'country_3', 'country_7',
       'country_8', 'country_5', 'country_1', 'country_12', 'country_15',
       'country_18', 'country_17', 'country_6', 'country_10', 'country_20',
       'country_2', 'country_19', 'country_11', 'country_13', 'country_0',
       'country_14']

In [6]:
class CFDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.csv_file = csv_file
        self.transform = transform
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        
        
    def __len__(self):
        return len(self.csv_file)
    
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        features = self.csv_file[self.feature_cols].iloc[idx]
        features = torch.FloatTensor(features)
        try:
            target = self.csv_file[self.target_cols].iloc[idx]
            target = torch.tensor(target)
            # train_step
        except:
            target = self.csv_file["id"].iloc[idx]
            # test_step
        
        if self.transform:
            features = self.transform(features)
            
        return features, target

In [7]:
class CFDataModule(pl.LightningDataModule):
    def __init__(self, train, validation, transform, split_rate, batch_size, num_workers):
        super().__init__()
        self.csv_file = train
        self.transform = transform
        self.split_rate = split_rate
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.validation = validation
        

    def setup(self, stage=None):
        self.train_dataset = CFDataset(csv_file=self.csv_file, transform=self.transform)
        self.val_dataset = CFDataset(csv_file=self.validation, transform=self.transform)
            
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          pin_memory=True)
    
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          pin_memory=True)

In [8]:
class CFModule(pl.LightningModule):
    def __init__(self, num_features, num_classes, trial):
        super(CFModule, self).__init__()
        self.layers = []
        self.dense_layers = []
        self.dropouts = []

        # We optimize the number of layers, hidden units in each layer and dropouts.
        n_layers = trial.suggest_int("n_layers", 2, 4)
        dropout = trial.suggest_float("dropout", 0.2, 0.5)
        input_dim = num_features
        for i in range(n_layers):
            output_dim = trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True)
            self.layers.append(nn.BatchNorm1d(input_dim))
            self.dropouts.append(nn.Dropout(dropout))
            self.dense_layers.append(nn.utils.weight_norm(nn.Linear(input_dim, output_dim)))
            input_dim = output_dim

        self.layers.append(nn.Linear(input_dim, num_classes))

        # Assigning the layers as class variables (PyTorch requirement).
        # Parameters of a layer are returned when calling model.parameters(),
        # only if the layer is a class variable. Thus, assigning as class
        # variable is necessary to make the layer parameters trainable.
        for idx, layer in enumerate(self.layers):
            setattr(self, "fc{}".format(idx), layer)

        # Assigning the dropouts as class variables (PyTorch requirement), for
        # the same reason as above.
        for idx, dropout in enumerate(self.dropouts):
            setattr(self, "drop{}".format(idx), dropout)
            
        for idx, dense in enumerate(self.dense_layers):
            setattr(self, "dense{}".format(idx), dense)
            
    
    def forward(self, x):
        for layer, dropout, dense_layer in zip(self.layers, self.dropouts, self.dense_layers):
            x = layer(x)
            x = dropout(x)
            x = F.relu(dense_layer(x))
        x = self.layers[-1](x)
        
        return F.sigmoid(x)
    
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        # you should define log as {"tag_name/log_name"}
        tensorboard_logs = {'train/train_loss': loss, "train/train_acc": acc}
        return {"loss": loss, "acc": acc, "logs": tensorboard_logs, "progress_bar": tensorboard_logs}
    
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        logs = {"val_loss": loss, "val_acc": acc}
        self.log("vall_loss", loss)
        return {"val_loss": loss, "val_acc": acc, "progress_bar": logs}

    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([torch.tensor(x['val_acc']) for x in outputs]).mean()
        tensorboard_logs = {'val/avg_loss': avg_loss, "val/avg_acc": avg_acc}
        #print(f"val_loss: {avg_loss}, val_acc: {avg_acc}")
        # you should call back as name "val_loss" to using the Early-Stopping
        return {'val_loss': avg_loss, "val_acc": avg_acc, 'log': tensorboard_logs}
    
    
    def configure_optimizers(self):
        optimizer = optimizers.Adam(self.parameters(), 
                                    lr=0.001, betas=(0.9,0.999),
                                    eps=1e-08, weight_decay=0, amsgrad=False)
        scheduler = {"scheduler": 
                     optimizers.lr_scheduler.CosineAnnealingLR(
                        optimizer, T_max=10),
                    "interval": "epoch",
                    "monitor": "val_loss"}
        return [optimizer], [scheduler]
    
    
    def criterion(self, pred, t):
        #pred = pred.view(-1)
        pred = pred.float()
        t = t.float()
        return F.binary_cross_entropy(input=pred, target=t)
    
    def metric(self, pred, t):
        t = t.to('cpu')
        try:
            pred = pred.detach().numpy()
        except:
            pred = pred.to('cpu')
        pred = np.where(pred<0.5, 0, 1)
        return f1_score(y_true=t, y_pred=pred, average='binary', sample_weight=None, zero_division='warn')

In [9]:
class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [10]:
def objective(trial):
    
    # data module config
    seed = 123456
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    for fold, (t_idx, v_idx) in enumerate(cv.split(train_data)):
        train_set = train_data.loc[t_idx]
        val_set = train_data.loc[v_idx]
        
    transform = None
    split_rate = 0.8
    batch_size = 256*4
    num_workers = 4
    PERCENT_VALID_EXAMPLES = 0.7
    EPOCHS = 15
    
    # model config
    num_input = len(feature_cols)
    num_classes = 1
    
    cf = CFDataModule(train_set, val_set, transform, split_rate, batch_size, num_workers)
    model = CFModule(num_features=num_input, num_classes=num_classes, trial=trial)
    
    
    
    # Filenames for each trial must be made unique in order to access each checkpoint.
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss"
    )

    # The default logger in PyTorch Lightning writes to event files to be consumed by
    # TensorBoard. We don't use any logger here as it requires us to implement several abstract
    # methods. Instead we setup a simple callback, that saves metrics from each validation step.
    metrics_callback = MetricsCallback()
    trainer = pl.Trainer(
        logger=False,
        limit_val_batches=PERCENT_VALID_EXAMPLES,
        checkpoint_callback=checkpoint_callback,
        max_epochs=EPOCHS,
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback, PyTorchLightningPruningCallback(trial, monitor="val_loss")],
    )
    
    trainer.fit(model, cf)

    return metrics_callback.metrics[-1]["val_loss"].item()

In [11]:
if __name__ == "__main__":
    
    DIR = os.getcwd()
    MODEL_DIR = os.path.join(DIR, "result")
    
    pruner = optuna.pruners.NopPruner()

    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(objective, n_trials=100, timeout=int(1.5*60*60))

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    shutil.rmtree(MODEL_DIR)

[I 2020-12-31 01:44:09,044] A new study created in memory with name: no-name-ce5e0ed7-6e8f-45e1-8184-1f178332819e
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 134   
1 | fc1    | BatchNorm1d | 110   
2 | fc2    | Linear      | 24    
3 | drop0  | Dropout     | 0     
4 | drop1  | Dropout     | 0     
5 | dense0 | Linear      | 3.8 K 
6 | dense1 | Linear      | 1.3 K 


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


[I 2020-12-31 01:49:16,549] Trial 0 finished with value: 0.5693934559822083 and parameters: {'n_layers': 2, 'dropout': 0.3095289916017212, 'n_units_l0': 55, 'n_units_l1': 23}. Best is trial 0 with value: 0.5693934559822083.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name   | Type        | Params
----------------------------------------
0  | fc0    | BatchNorm1d | 134   
1  | fc1    | BatchNorm1d | 58    
2  | fc2    | BatchNorm1d | 40    
3  | fc3    | BatchNorm1d | 108   
4  | fc4    | Linear      | 29    
5  | drop0  | Dropout     | 0     
6  | drop1  | Dropout     | 0     
7  | drop2  | Dropout     | 0     
8  | drop3  | Dropout     | 0     
9  | dense0 | Linear      | 2.0 K 
10 | dense1 | Linear      | 620   
11 | dense2 | Linear      | 1.2 K 
12 | dense3 | Linear      | 1.6 K 


[I 2020-12-31 01:54:12,278] Trial 1 finished with value: 0.6052200198173523 and parameters: {'n_layers': 4, 'dropout': 0.2857492657001222, 'n_units_l0': 29, 'n_units_l1': 20, 'n_units_l2': 54, 'n_units_l3': 28}. Best is trial 0 with value: 0.5693934559822083.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 134   
1 | fc1    | BatchNorm1d | 22    
2 | fc2    | BatchNorm1d | 8     
3 | fc3    | Linear      | 59    
4 | drop0  | Dropout     | 0     
5 | drop1  | Dropout     | 0     
6 | drop2  | Dropout     | 0     
7 | dense0 | Linear      | 759   
8 | dense1 | Linear      | 52    
9 | dense2 | Linear      | 348   


...

[I 2020-12-31 03:03:24,948] Trial 15 finished with value: 0.576632022857666 and parameters: {'n_layers': 2, 'dropout': 0.4485481476448035, 'n_units_l0': 68, 'n_units_l1': 44}. Best is trial 11 with value: 0.5366407632827759.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 134   
1 | fc1    | BatchNorm1d | 220   
2 | fc2    | Linear      | 119   
3 | drop0  | Dropout     | 0     
4 | drop1  | Dropout     | 0     
5 | dense0 | Linear      | 7.6 K 
6 | dense1 | Linear      | 13.2 K


[I 2020-12-31 03:08:18,822] Trial 16 finished with value: 0.5391666889190674 and parameters: {'n_layers': 2, 'dropout': 0.3662293598551007, 'n_units_l0': 110, 'n_units_l1': 118}. Best is trial 11 with value: 0.5366407632827759.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 134   
1 | fc1    | BatchNorm1d | 166   
2 | fc2    | Linear      | 49    
3 | drop0  | Dropout     | 0     
4 | drop1  | Dropout     | 0     
5 | dense0 | Linear      | 5.7 K 
6 | dense1 | Linear      | 4.1 K 


[I 2020-12-31 03:13:09,513] Trial 17 finished with value: 0.5499637722969055 and parameters: {'n_layers': 2, 'dropout': 0.35518653468982336, 'n_units_l0': 83, 'n_units_l1': 48}. Best is trial 11 with value: 0.5366407632827759.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type        | Params
---------------------------------------
0 | fc0    | BatchNorm1d | 134   
1 | fc1    | BatchNorm1d | 44    
2 | fc2    | Linear      | 128   
3 | drop0  | Dropout     | 0     
4 | drop1  | Dropout     | 0     
5 | dense0 | Linear      | 1.5 K 
6 | dense1 | Linear      | 3.0 K 


[I 2020-12-31 03:18:01,813] Trial 18 finished with value: 0.5891623497009277 and parameters: {'n_layers': 2, 'dropout': 0.3625518790724301, 'n_units_l0': 22, 'n_units_l1': 127}. Best is trial 11 with value: 0.5366407632827759.



Number of finished trials: 19
Best trial:
  Value: 0.5366407632827759
  Params: 
    n_layers: 2
    dropout: 0.4013092486547058
    n_units_l0: 120
    n_units_l1: 104
